In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import ARDRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor

from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import MultiTaskLasso

In [6]:
from sklearn.metrics import mean_squared_error

In [6]:
from tqdm import tqdm

In [7]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [8]:
def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [9]:
train.head(5)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [10]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [11]:
FEATURE_COLUMNS = ['full_text']
TARGET_COLUMNS = ['cohesion', 'syntax','vocabulary', 'phraseology', 'grammar','conventions']

In [12]:
X, Y = train[FEATURE_COLUMNS], train[TARGET_COLUMNS]


In [13]:
kfold = KFold(n_splits=5)

#### LinReg
Mean score: .55

In [26]:
predictions = []
losses = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    losses.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean loss: {np.mean(losses):.3f}")
    

Fold #0: 0.5541318247074661
Fold #1: 0.5537700023456712
Fold #2: 0.5470261009151448
Fold #3: 0.5444073250115814
Fold #4: 0.552034364850683
********************************************************************************

	Mean loss: 0.550


In [30]:
# with rounding 
predictions = []
losses = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred*2) / 2
    score_ = mcrmse(Y_test, y_pred)
    losses.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean loss: {np.mean(losses):.3f}")
    

Fold #0: 0.5709403512789716
Fold #1: 0.5726226508335558
Fold #2: 0.5635645363980998
Fold #3: 0.5632515029920215
Fold #4: 0.5698789714147964
********************************************************************************

	Mean loss: 0.568


#### ElasticNet

In [14]:
predictions = []
losses = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(ElasticNet())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    losses.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean loss: {np.mean(losses):.3f}")

Metal device set to: Apple M1
Fold #0: 0.6528782028635198


2022-09-14 16:32:46.981353: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-14 16:32:46.982955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


KeyboardInterrupt: 

#### Kernel ridge 

In [15]:
predictions = []
losses = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(KernelRidge())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    losses.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean loss: {np.mean(losses):.3f}")

Fold #0: 0.6930777265189026


KeyboardInterrupt: 

In [ ]:
# Bayesian ridge takes forever 
# I should try ridge as well 
# 

#### Submission

In [31]:
preds_mean = np.mean(predictions, axis = 0)
submission = pd.DataFrame()
submission.loc[:,'text_id'] = test.text_id
submission.loc[:, TARGET_COLUMNS] = preds_mean

# submission.loc[:, TARGET_COLUMNS] = (submission.loc[:, TARGET_COLUMNS] * 2).round() / 2
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.066927,2.759472,3.144001,3.051319,2.492089,2.823449
1,000BAD50D026,2.447473,2.554104,2.579586,2.329913,2.287815,2.528216
2,00367BB2546B,3.582943,3.397358,3.529512,3.458739,3.600916,3.366636


In [24]:
submission.to_csv('submission.csv', index=False)

In [25]:
# Score .54 LOL